In [1]:
import nltk
from sklearn.metrics import classification_report, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
import torch
import torch.nn as nn

#from torch_model_base import TorchModelBase
#from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from torch_rnn_classifier import TorchRNNDataset, TorchRNNClassifier, TorchRNNModel
import utils

import pandas as pd
from collections import Counter

In [2]:
# refresh torch rnn classifier:
import importlib
import torch_rnn_classifier
importlib.reload(torch_rnn_classifier)
from torch_rnn_classifier import TorchRNNDataset

In [3]:
import json

In [188]:
with open('annotations2.jsonl') as jsonl_file:
    # note: after running data-preprocessing.ipynb this file already has token-level labels
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]

In [189]:
# now get data into format that TorchRNN expects:
X=[] 
y=[]
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    auxX = []
    auxy = []
    if annot[j]['spans']!=[]: # are there annot for this example?
        for i in range(0,len(a)):
            #token_element = (a[i]['text'],a[i]['label'])
            auxX.append(a[i]['text'])
            auxy.append(a[i]['label'])
        X.append(auxX)
        y.append(auxy)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_test, y_train, y_test = X[:120], X[120:], y[:120], y[120:]
vocab = sorted({w for seq in X_train for w in seq}) + ["$UNK"]

In [65]:
# reload vsm module
import torch_rnn_classifier, torch_model_base
import importlib
importlib.reload(torch_model_base)
importlib.reload(torch_rnn_classifier)
from torch_model_base import TorchModelBase
from torch_rnn_classifier import TorchRNNClassifier, TorchRNNModel, TorchRNNDataset

In [5]:
class TorchRNNSequenceLabeler(TorchRNNClassifier):

    def build_graph(self): # uses this build_graph instead of TorchRNNClassifier.build_graph
        print("here0")
        rnn = TorchRNNModel(
            vocab_size=len(self.vocab),
            embedding=self.embedding,
            use_embedding=self.use_embedding,
            embed_dim=self.embed_dim,
            rnn_cell_class=self.rnn_cell_class,
            hidden_dim=self.hidden_dim,
            bidirectional=self.bidirectional,
            freeze_embedding=self.freeze_embedding)
        print("here02")
        model = TorchSequenceLabeler( # this defines self.model
            rnn=rnn,
            output_dim=self.n_classes_)
        self.embed_dim = rnn.embed_dim
        return model

    def build_dataset(self, X, y=None):
        START_TAG = "<START>"
        STOP_TAG = "<STOP>"
        X, seq_lengths = self._prepare_sequences(X) # converts tokens into tokenIds
        if y is None:
            return TorchRNNDataset(X, seq_lengths)
        else:
            # These are the changes from a regular classifier. All
            # concern the fact that our labels are sequences of labels.
            self.classes_ = sorted({x for seq in y for x in seq})
            self.classes_.append(START_TAG) # add start and stop tags
            self.classes_.append(STOP_TAG)
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            # `y` is a list of tensors of different length. Our Dataset
            # class will turn it into a padding tensor for processing.
            y = [torch.tensor([class2index[label] for label in seq])
                 for seq in y] # converts labels to indices
            return TorchRNNDataset(X, seq_lengths, y)

    def predict_proba(self, X):
        seq_lengths = [len(ex) for ex in X]
        # The base class does the heavy lifting:
        preds = self._predict(X)
        # Trim to the actual sequence lengths:
        preds = [p[: l] for p, l in zip(preds, seq_lengths)]
        # Use `softmax`; the model doesn't do this because the loss
        # function does it internally.
        probs = [torch.softmax(seq, dim=1) for seq in preds]
        return probs

    def predict(self, X):
        probs = self.predict_proba(X)
        return [[self.classes_[i] for i in seq.argmax(axis=1)] for seq in probs] # seq.argmax(axis=1) gives index of col that maximizes softmax prob
        # see difference vs TorchRNNClassifier.predict

    def score(self, X, y):
        preds = self.predict(X)
        flat_preds = [x for seq in preds for x in seq]
        flat_y = [x for seq in y for x in seq]
        return utils.safe_macro_f1(flat_y, flat_preds)
    

    
    
class TorchSequenceLabeler(nn.Module): # no self.hidden_layer or self.classifier_activation as TorchRNNClassifierModel
    def __init__(self, rnn, output_dim):
        print("here021")
        super().__init__()
        self.rnn = rnn
        self.output_dim = output_dim
        if self.rnn.bidirectional:
            self.classifier_dim = self.rnn.hidden_dim * 2
        else:
            self.classifier_dim = self.rnn.hidden_dim
        self.classifier_layer = nn.Linear(
            self.classifier_dim, self.output_dim)

    def forward(self, X, seq_lengths): # X is (noExsInBatch,MaxLen)=(108,117), seq_lengths is the number of tokens in each example in each batch
        # this is the forward method of self.model
        print("here2")
        outputs, state = self.rnn(X, seq_lengths) # X is (batchSize, maxLen of exs in batch); outputs is (noTokensInEx,hiddDim), state is ((batch_size,1,hiddDim),(batch_size,1,hiddDim)) = (finalHiddState,finalCellState) 
        #print(outputs.data.shape)
        #print(state[0].data.shape)
        #print(state[1].data.shape)
        outputs, seq_length = torch.nn.utils.rnn.pad_packed_sequence(
            outputs, batch_first=True) # outputs is (batchSize,MaxLen of examples in batch,hidden_dim); seq_length is noTokenInEx for each ex in batch
        #print(outputs.data.shape)
        #print(seq_length)
        logits = self.classifier_layer(outputs) # this is an FCL from hidden_dim to output_dim (NoLabelClasses)
        # logits are (108,117,12) or (1,11,5) = (batchSize,MaxLen of examples in batch,noLabelClasses) noLabelClasses include Start + End
        # During training, we need to swap the dimensions of logits
        # to accommodate `nn.CrossEntropyLoss`:
        print(logits)
        if self.training:
            return logits.transpose(1, 2) # transpose dimensions 1 and 2 w/ each other (3d array) # outputs (108,12,117) or (1,5,11)
        else:
            return logits

In [7]:
X_train = ["the wall street journal reported today that apple corporation made money".split(),"georgia tech is a university in georgia".split()]
y_train = ["B I I I O O O B I O O".split(),"B I O O O O B".split()]
vocab = sorted({w for seq in X_train for w in seq}) + ["$UNK"]

In [8]:
seq_mod = TorchRNNSequenceLabeler(
    vocab,
    early_stopping=True,
    eta=0.001)

In [9]:
%time _ = seq_mod.fit(X_train, y_train)

here00
here0
here01
here02
here021
batch1
here2
here21


Finished epoch 10 of 1000; error is 1.3484532833099365

tensor([[[-0.3627, -0.0571,  0.1751, -0.1338,  0.0219],
         [-0.0976,  0.0181,  0.0332, -0.0325,  0.1119],
         [-0.2014,  0.0605,  0.1298, -0.0567,  0.0970],
         [-0.2015,  0.1037,  0.0392, -0.0535,  0.0768],
         [-0.0959,  0.0611,  0.0748, -0.1305,  0.1798],
         [-0.1551,  0.0170,  0.0991, -0.1830,  0.0736],
         [-0.0786,  0.0709,  0.0116, -0.0523, -0.1000],
         [-0.1403,  0.1220, -0.0602, -0.0288,  0.1099],
         [-0.2416,  0.1540,  0.1170,  0.0194,  0.1142],
         [-0.2535, -0.0141,  0.0446,  0.1006,  0.1224],
         [-0.1576, -0.1045,  0.1067,  0.0832,  0.1514]]], device='cuda:0',
       grad_fn=<AddBackward0>)
here2
here21
tensor([[[-0.1249, -0.0782,  0.0023, -0.0520,  0.1311],
         [-0.0366,  0.0341,  0.0813, -0.1528,  0.0561],
         [-0.0623,  0.1088,  0.0040, -0.1438,  0.1385],
         [-0.1064,  0.0313,  0.1009, -0.0774,  0.0317],
         [-0.2064,  0.0638,  0.1199,  0.0804,  0.0294],
         [-0.1743,  0.0478,  0.0579,  0.1

Stopping after epoch 17. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 1.1498440504074097

batch1
here2
here21
tensor([[[-0.2055,  0.0330,  0.1358, -0.2117, -0.0419],
         [-0.1146,  0.3050,  0.1270, -0.1609, -0.0685],
         [-0.2174,  0.3682,  0.2005, -0.1287, -0.0615],
         [-0.2659,  0.4222,  0.1476, -0.2032, -0.1413],
         [-0.2114,  0.1298,  0.4285, -0.3082, -0.0199],
         [-0.2997, -0.0067,  0.5334, -0.3325, -0.0855],
         [-0.1073, -0.0274,  0.3898, -0.2129, -0.2400],
         [ 0.0797,  0.1612,  0.1368, -0.1711,  0.0008],
         [-0.2606,  0.3549,  0.3965, -0.1893, -0.1035],
         [-0.3843,  0.0578,  0.5206, -0.1210, -0.0546],
         [-0.2773, -0.1194,  0.6506, -0.0980, -0.0408]]], device='cuda:0',
       grad_fn=<AddBackward0>)
here2
here21
tensor([[[-0.0833, -0.0758,  0.0173, -0.0734,  0.1063],
         [-0.0720,  0.0183,  0.2113, -0.2265, -0.0212],
         [-0.0423,  0.1534,  0.0486, -0.1658,  0.0914],
         [-0.1369,  0.0838,  0.2160, -0.1259, -0.0410],
         [-0.2070,  0.1469,  0.2619, -0.0441, -0.0952],
         [-0.1904,  0

In [99]:
from torchcrf import CRF

class TorchCRFSequenceLabeler(TorchRNNClassifier):

    def build_graph(self): # uses this build_graph instead of TorchRNNClassifier.build_graph
        print("here0")
        rnn = TorchRNNModel(
            vocab_size=len(self.vocab),
            embedding=self.embedding,
            use_embedding=self.use_embedding,
            embed_dim=self.embed_dim,
            rnn_cell_class=self.rnn_cell_class,
            hidden_dim=self.hidden_dim,
            bidirectional=self.bidirectional,
            freeze_embedding=self.freeze_embedding)
        print("here02")
        model = CRF(num_tags)
       # model = TorchSequenceLabeler( # this defines self.model
       #     rnn=rnn,
       #     output_dim=self.n_classes_)
       # self.embed_dim = rnn.embed_dim
        return model

    def build_dataset(self, X, y=None):
        START_TAG = "<START>"
        STOP_TAG = "<STOP>"
        X, seq_lengths = self._prepare_sequences(X) # converts tokens into tokenIds
        if y is None:
            return TorchRNNDataset(X, seq_lengths)
        else:
            # These are the changes from a regular classifier. All
            # concern the fact that our labels are sequences of labels.
            self.classes_ = sorted({x for seq in y for x in seq})
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(2,self.n_classes_+2))) # start at id=2
            class2index[STOP_TAG]=0    # add start and stop tags (note: stop needs to be 0 as that is default for padding in collate_fn)
            class2index[START_TAG]=1 
            print(class2index)
            # `y` is a list of tensors of different length. Our Dataset
            # class will turn it into a padding tensor for processing.
            y = [torch.tensor([class2index[label] for label in seq])
                 for seq in y] # converts labels to indices
            return TorchRNNDataset(X, seq_lengths, y)

    def predict_proba(self, X):
        seq_lengths = [len(ex) for ex in X]
        # The base class does the heavy lifting:
        preds = self._predict(X)
        # Trim to the actual sequence lengths:
        preds = [p[: l] for p, l in zip(preds, seq_lengths)]
        # Use `softmax`; the model doesn't do this because the loss
        # function does it internally.
        probs = [torch.softmax(seq, dim=1) for seq in preds]
        return probs

    def predict(self, X):
        probs = self.predict_proba(X)
        return [[self.classes_[i] for i in seq.argmax(axis=1)] for seq in probs] # seq.argmax(axis=1) gives index of col that maximizes softmax prob
        # see difference vs TorchRNNClassifier.predict

    def score(self, X, y):
        preds = self.predict(X)
        flat_preds = [x for seq in preds for x in seq]
        flat_y = [x for seq in y for x in seq]
        return utils.safe_macro_f1(flat_y, flat_preds)
    


In [100]:
# Following converts words to indices and pads sequences
seq_mod = TorchCRFSequenceLabeler(
    vocab,
    early_stopping=True,
    eta=0.001)

In [12]:
X_train = ["the wall street journal reported today that apple corporation made money".split(),"georgia tech is a university in georgia".split()]
y_train = ["B I I I O O O B I O O".split(),"B I O O O O B".split()]

In [119]:
torch.manual_seed(1)
dataset = seq_mod.build_dataset(X_train, y_train) 
dataloader = seq_mod._build_dataloader(dataset, shuffle=False) 
num_tags = 5
model = CRF(num_tags)
for batch_num, batch in enumerate(dataloader, start=1):
    x=batch[0]   
    print(x)
    seq_length=batch[1]
    print(seq_length)
    y=batch[2] 
    print(y)
    print(seq_length)
batch_size=1
emissions = torch.randn(batch_size, max(seq_length), num_tags)
print(emissions[0])
y=torch.tensor(y[1]).view(1,-1)
print(y.shape)
print(model(emissions,y)) # computes log likelihood
#model.decode(emissions)
a=model.transitions
print(a)
print(model.decode(emissions))

{'B': 2, 'I': 3, 'O': 4, '<STOP>': 0, '<START>': 1}
tensor([[13, 16, 10,  6,  9, 14, 12,  1,  2,  7,  8],
        [ 3, 11,  5,  0, 15,  4,  3,  0,  0,  0,  0]])
tensor([11,  7])
tensor([[2, 3, 3, 3, 4, 4, 4, 2, 3, 4, 4],
        [2, 3, 4, 4, 4, 4, 2, 0, 0, 0, 0]])
tensor([11,  7])
tensor([[ 0.0457,  0.1530, -0.4757, -1.8821, -0.7765],
        [ 2.0242, -0.0865,  0.0981, -1.0373,  1.5748],
        [-0.6298,  2.4070,  0.2786,  0.2468,  1.1843],
        [-0.7282,  0.4415,  1.1651,  2.0154,  0.2152],
        [-0.5242, -1.8034, -1.3083,  0.4533, -0.8696],
        [-3.3312, -0.7479,  1.1173,  0.2981,  0.1099],
        [-0.6463,  0.4285, -0.0075,  1.6734,  0.0103],
        [ 0.9837,  0.8793, -1.4504, -1.1802,  1.3075],
        [-1.1628,  0.1196, -0.1631,  0.6614,  1.1899],
        [ 0.8165, -0.9135, -0.3538,  0.7639, -0.5890],
        [-0.7636,  1.3352,  0.6043, -0.1034, -0.1512]])
torch.Size([1, 11])
tensor(-23.6227, grad_fn=<SumBackward0>)
Parameter containing:
tensor([[ 0.0277,  0.0049,  0

C:\Users\vasco\AppData\Local\Temp/ipykernel_17200/1289059235.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(y[1]).view(1,-1)


In [149]:
# this is where I matched my first model score (i.e. log likelihood of crf)
torch.manual_seed(1)
seq_length = 1  # maximum sequence length in a batch
batch_size = 1  # number of samples in the batch
num_tags=2
model = CRF(num_tags,batch_first=True)
#emissions = torch.randn(batch_size, seq_length, num_tags)
#tags = torch.tensor([[0, 2, 3], [1, 4, 1]], dtype=torch.long)  # (batch_size, seq_length)
emissions = torch.randn(1, seq_length, num_tags)

tags = torch.tensor([[1]], dtype=torch.long)  
print(model(emissions, tags))
print(emissions)
print(tags)
print(model.transitions)
print(model.start_transitions)
print(model.end_transitions)

tensor(-0.5731, grad_fn=<SumBackward0>)
tensor([[[-0.4519, -0.1661]]])
tensor([[1]])
Parameter containing:
tensor([[-0.0941,  0.0600],
        [-0.0206,  0.0509]], requires_grad=True)
Parameter containing:
tensor([ 0.0515, -0.0441], requires_grad=True)
Parameter containing:
tensor([-0.0194,  0.0469], requires_grad=True)


In [176]:
# add mask vector so as to not to consider padding part of sequences
# e.g. want to mask last zero of 2nd obs
torch.manual_seed(1)
seq_length = 3  # maximum sequence length in a batch
batch_size = 2  # number of samples in the batch
num_tags=5
model = CRF(num_tags,batch_first=True)
emissions = torch.randn(batch_size, seq_length, num_tags)
tags = torch.tensor([[1, 2, 3], [1, 4, 0]], dtype=torch.long)  # (batch_size, seq_length)
mask = torch.tensor([[1, 1, 1], [1, 1, 0]], dtype=torch.uint8) # i.e. mask 3rd token of 2nd example in batch
# 1.
print(model(emissions, tags, mask=mask)) # model log likelihood
# 2.
print(model.decode(emissions,mask=mask)) # most likely tag sequences
# 3. inference:
tags_test = torch.tensor([[1, 2, 3]], dtype=torch.long)  # (batch_size, seq_length)
print(model.forward(emissions[0].unsqueeze(dim=0),tags_test)) # returns log likelihood of test tag sequence (larger no. means more likely)
# note: need to use torch array w/ same no. of examples as tags_test

tensor(-8.3568, grad_fn=<SumBackward0>)
[[3, 4, 3], [4, 1]]
tensor(-2.7487, grad_fn=<SumBackward0>)


In [377]:
# note: this isn't exactly correct as training examples are shuffled (esp. max len of the smaller 12 ex batch is != 92)
auxMax=0
x_max_idx=108
for i in range(0,min(len(X_train),x_max_idx)):
    if len(X_train[i])>auxMax:
        auxMax=len(X_train[i])
print(auxMax)
auxMax2=0
x_min_idx=109
for i in range(max(0,x_min_idx),len(X_train)):
    if len(X_train[i])>auxMax2:
        auxMax2=len(X_train[i])
print(auxMax2)

117
92


In [107]:
y_pred = seq_mod.predict(X_test)
print(y_test[0])
print(y_pred[0])

['ORT', 'O', 'O', 'STRASSE', 'STRASSE', 'O', 'ORT', 'O', 'O', 'FLAECHE', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'O', 'O', 'QMPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERTRAG', 'DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERBUECHERUNG', 'DATUM_VERBUECHERUNG', 'O']
['KAEUFER', 'KAEUFER', 'O', 'O', 'KAEUFER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [108]:
labels=seq_mod.classes_
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

In [109]:
# unfold all our data - NOTE: this means we don't care about per sentence results. 
# i.e. each classification is worth same regardless of sentence in which it occurs
y_test_unfold = [y for element in y_test for y in element]
y_pred_unfold = [y for element in y_pred for y in element]

In [126]:
print(y_test_unfold[:10])
print(y_pred_unfold[:10])

['ORT', 'O', 'O', 'STRASSE', 'STRASSE', 'O', 'ORT', 'O', 'O', 'FLAECHE']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [82]:
# convert y_test and y_pred into binary formats
#from sklearn.preprocessing import MultiLabelBinarizer

In [110]:
print(classification_report(
    y_test_unfold, y_pred_unfold, labels=sorted_labels, digits=3
))

                     precision    recall  f1-score   support

                  O      0.808     0.888     0.846       643
            KAEUFER      0.070     0.278     0.112        18
DATUM_VERBUECHERUNG      0.000     0.000     0.000        25
      DATUM_VERTRAG      1.000     0.037     0.071        27
         VERKAEUFER      0.333     0.042     0.074        24
   TERRASSENGROESSE      0.000     0.000     0.000         5
        GESAMTPREIS      0.000     0.000     0.000        11
            FLAECHE      0.000     0.000     0.000        15
           IMMO_TYP      0.000     0.000     0.000        19
            QMPREIS      0.000     0.000     0.000        10
                ORT      0.000     0.000     0.000        26
            STRASSE      0.118     0.125     0.121        16

           accuracy                          0.691       839
          macro avg      0.194     0.114     0.102       839
       weighted avg      0.664     0.691     0.657       839



Now try with leading "B-" and "I-"

In [83]:
########## ONLY RUN IF WE WANT TO ADD LEADING "B-" / "I-" TO CLASS LABEL
# now use above code and loop through all items of annot list:
# addLeading=1 for "Yes" (i.e. add leading "B-","I-" to annot); 0 for "No" (i.e. add labels to annot simply as they are)
addLeading = 1

if addLeading == 1:
    for j in range(0,len(annot)):
        a = annot[j]
        # select list of dict of tokens w/ annnotations and add column w/ no. of words to each dict:
        b = a['spans']
        # add noWords to b dict. note: b is list of dicts w/ annotations; tokens not on this list don't have annotations
        if b!=[]: #i.e. only try to add annotations to tokens if there are annotations to begin with
            #print(b)
            for i in range(0,len(annot[j]['tokens'])):
                    # now break-up label into 1st occurrence (leading "B-") and subsequent occurrences (leading "I-") (only for non "O"'s)
                    if annot[j]['tokens'][i]['label'] != "O":
                        if i==0:
                            annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label']
                        else: 
                            if annot[j]['tokens'][i]['label'] == annot[j]['tokens'][i-1]['label'][2:]: # need to remove the leading "B-" that we had already been added to c[i-1]
                                annot[j]['tokens'][i]['label'] = "I-" + annot[j]['tokens'][i]['label']
                            else:
                                annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label'] 

In [84]:
# now get data into format that TorchRNN expects:
X=[] 
y=[]
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    auxX = []
    auxy = []
    if annot[j]['spans']!=[]: # are there annot for this example?
        for i in range(0,len(a)):
            #token_element = (a[i]['text'],a[i]['label'])
            auxX.append(a[i]['text'])
            auxy.append(a[i]['label'])
        X.append(auxX)
        y.append(auxy)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_test, y_train, y_test = X[:120], X[120:], y[:120], y[120:]
vocab = sorted({w for seq in X_train for w in seq}) + ["$UNK"]

In [89]:
print(X_train[0])

['DORNBIRN', 'In', 'der', 'Schulgasse', 'in', 'Dornbirn', 'hat', 'eine', '71,93', 'Quadratmeter', 'große', 'Wohnung', 'für', 'einen', 'Quadratmeterpreis', 'von', '5533,71', 'Euro', 'den', 'Besitzer', 'gewechselt', '.', 'Dieser', 'beinhaltet', 'auch', 'einen', 'Pkw-Abstellplatz', '.', 'Käufer', 'der', 'Wohnung', 'mit', '9,86', 'Quadratmetern', 'Terrasse', 'ist', 'die', 'ValLiLean', 'Beteiligungs-', 'und', 'Immobilienverwaltungs', 'GmbH', 'Beim', 'Verkäufer', 'handelt', 'es', 'sich', 'um', 'die', 'Karrenblick', 'Projekt', 'GmbH', ' ', 'Der', 'Kaufpreis', 'liegt', 'bei', '398.040', 'Euro', '.', 'Unterzeichnet', 'wurde', 'der', 'Kaufvertrag', 'am', '18.', 'September', '.', 'Die', 'Verbücherung', 'datiert', 'mit', 'Oktober', '2020', '.', '.', '.']


In [85]:
%time _ = seq_mod.fit(X_train, y_train)
y_pred = seq_mod.predict(X_test)
print(y_test[0])
print(y_pred[0])

Stopping after epoch 35. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 2.249159574508667

Wall time: 1.78 s
['B-ORT', 'O', 'O', 'B-STRASSE', 'I-STRASSE', 'O', 'B-ORT', 'O', 'O', 'B-FLAECHE', 'O', 'O', 'B-IMMO_TYP', 'O', 'O', 'O', 'O', 'B-QMPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'B-GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'O', 'B-DATUM_VERBUECHERUNG', 'I-DATUM_VERBUECHERUNG', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [90]:
labels=seq_mod.classes_
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

In [91]:
# unfold all our data - NOTE: this means we don't care about per sentence results. 
# i.e. each classification is worth same regardless of sentence in which it occurs
y_test_unfold = [y for element in y_test for y in element]
y_pred_unfold = [y for element in y_pred for y in element]

In [92]:
print(classification_report(
    y_test_unfold, y_pred_unfold, labels=sorted_labels, digits=3
))

                       precision    recall  f1-score   support

                    O      0.773     0.988     0.867       643
B-DATUM_VERBUECHERUNG      0.000     0.000     0.000        13
I-DATUM_VERBUECHERUNG      0.000     0.000     0.000        12
      B-DATUM_VERTRAG      0.000     0.000     0.000        13
      I-DATUM_VERTRAG      0.000     0.000     0.000        14
            B-FLAECHE      0.000     0.000     0.000        15
            I-FLAECHE      0.000     0.000     0.000         0
        B-GESAMTPREIS      0.000     0.000     0.000        11
        I-GESAMTPREIS      0.000     0.000     0.000         0
           B-IMMO_TYP      0.000     0.000     0.000        19
           I-IMMO_TYP      0.000     0.000     0.000         0
            B-KAEUFER      0.000     0.000     0.000        10
            I-KAEUFER      0.000     0.000     0.000         8
                B-ORT      0.300     0.115     0.167        26
            B-QMPREIS      0.000     0.000     0.000  

C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Remove "B-" and "I-" (in case they are present in labels)

In [119]:
for j in range(0,len(annot)):
    a = annot[j]
    b = a['spans']
    if b!=[]: #i.e. only try to add annotations to tokens if there are annotations to begin with
        for i in range(0,len(annot[j]['tokens'])):
                if annot[j]['tokens'][i]['label'] != "O":
                    if annot[j]['tokens'][i]['label'][:2]=="B-" or annot[j]['tokens'][i]['label'][:2]=="I-":
                        annot[j]['tokens'][i]['label']=annot[j]['tokens'][i]['label'][2:]

Try bi-directional LSTM

In [149]:
seq_mod = TorchRNNSequenceLabeler(
    vocab,
    early_stopping=True,
    eta=0.001,
    bidirectional=True)

In [132]:
print(y_train[0])

['ORT', 'O', 'O', 'STRASSE', 'O', 'ORT', 'O', 'O', 'FLAECHE', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'O', 'O', 'QMPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'TERRASSENGROESSE', 'O', 'O', 'O', 'O', 'KAEUFER', 'KAEUFER', 'KAEUFER', 'KAEUFER', 'KAEUFER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'VERKAEUFER', 'VERKAEUFER', 'VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERTRAG', 'DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERBUECHERUNG', 'DATUM_VERBUECHERUNG', 'O', 'O', 'O']


In [150]:
%time _ = seq_mod.fit(X_train, y_train)

None


Stopping after epoch 18. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 2.1157665252685547

Wall time: 2.12 s


In [123]:
y_pred = seq_mod.predict(X_test)
print(y_test[0])
print(y_pred[0])

labels=seq_mod.classes_
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

# unfold all our data - NOTE: this means we don't care about per sentence results. 
# i.e. each classification is worth same regardless of sentence in which it occurs
y_test_unfold = [y for element in y_test for y in element]
y_pred_unfold = [y for element in y_pred for y in element]

print(classification_report(
    y_test_unfold, y_pred_unfold, labels=sorted_labels, digits=3
))

['ORT', 'O', 'O', 'STRASSE', 'STRASSE', 'O', 'ORT', 'O', 'O', 'FLAECHE', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'O', 'O', 'QMPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERTRAG', 'DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERBUECHERUNG', 'DATUM_VERBUECHERUNG', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'TERRASSENGROESSE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'TERRASSENGROESSE', 'TERRASSENGROESSE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
                     precision    recall  f1-score   support

                  O      0.760     0.893     0.821       643
            KAEUFER      0.000     0.000     0.000        18
DATUM_VERBUECHERUN